In [9]:
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras.utils import to_categorical
from sklearn import preprocessing
from collections import deque
import random
from numpy import newaxis
import sys


In [10]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
with open('/content/gdrive/MyDrive/Colab Notebooks/data/modified (1).csv', 'r') as z:
    df = pd.DataFrame(pd.read_csv(z))

df.set_index('TS',inplace=True)

In [12]:
seqlen=20
def prep(df):
  for col in df.columns:
    if col!= "Season":
      df[col]=df[col].pct_change()
      df.dropna(inplace=True)
      df[col]=preprocessing.scale(df[col].values)
  return df

df.dropna(inplace=True)
seq_d=[]
pre=deque(maxlen=seqlen)
for i in df.values:
  pre.append([n for n in i[:-1]])
  if len(pre)==seqlen:
    seq_d.append([np.array(pre),i[:-1]])

random.shuffle(seq_d)

df.drop(["Unnamed: 0"],axis=1,inplace=True)

In [13]:
df.head()
l=df.iloc[:,[0,1,2,4,5,6]]
l

,V1,V2,V3,W2,W3,Season
TS,,,,,,
1481779800,230.459834,232.246458,231.743727,61.896241,0.106471,1
1481794200,232.629286,235.258992,233.957315,64.306541,38.469765,1
1481797800,233.379785,234.766217,232.949642,12.996327,64.990385,1
1481862600,230.026178,230.905907,230.556791,0.713788,0.502634,1
1481866200,229.061954,232.196039,231.070804,36.306355,37.953893,1
...,...,...,...,...,...,...
1528120800,235.441763,236.377831,235.032985,105.633668,57.933859,2
1528124400,232.340915,233.185799,232.158858,204.173027,102.993996,2
1528128000,231.533768,231.922184,231.052265,164.771038,114.729862,2


In [14]:
train=df.iloc[:4435].copy()
valid=df.iloc[4435:].copy()
train.dropna(axis=0)
valid.dropna(axis=0)

valid_x=prep(valid)
valid_x.dropna(axis=0)
valid_y=prep(valid)
valid_y.dropna(axis=0)

valid_x=(valid_x).to_numpy()
valid_x=valid_x[:,newaxis,:]
print(valid_x.shape)

valid_y=(valid_y).to_numpy()
valid_y=valid_y[:,newaxis,:]
print(valid_y.shape)

train_x=prep(train)
train_x.dropna(axis=0)
train_x=train_x[:4423]
train_x=(train_x).to_numpy()
train_x=train_x[:,newaxis,:]
print(train_x.shape)

train_y=prep(train)
train_y.dropna(axis=0)
train_y=train_y[:4423]
train_y=(train_y).to_numpy()
train_y=train_y[:,newaxis,:]
print(train_y.shape)




(221, 1, 7)
(221, 1, 7)
(4423, 1, 7)
(4423, 1, 7)


In [ ]:

print(train_x.shape)
print(train_y.shape)

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=128,input_shape=(train_x.shape[1:]),return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.LSTM(units=64,input_shape=(train_x.shape[1:]),return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.LSTM(units=64,input_shape=(train_x.shape[1:])))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(units=64,input_shape=(train_x.shape[1:]),activation=tf.keras.layers.PReLU()))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1,activation='relu'))

opt=tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(optimizer=opt,loss='mae',metrics=['accuracy'])
model.fit(train_x,train_y,epochs=36,validation_data=(valid_x,valid_y))
model.summary()